<a href="https://www.kaggle.com/code/abdulaziz04/prediction-using-catboostregressor?scriptVersionId=101164069" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#Linear Regression Models from sklearn
from sklearn.linear_model import LinearRegression,SGDRegressor,ElasticNet,BayesianRidge
from lightgbm import LGBMRegressor
from xgboost.sklearn import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import KFold,cross_val_score


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
pd.set_option('max_rows',1000)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
X=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
xtest=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
sample=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')

In [ ]:
y=X['SalePrice']
ids=xtest['Id']


In [ ]:
X.isnull().sum()

In [ ]:
plt.figure(figsize=(40,40))
corr=X.corr()
dataplot = sns.heatmap(corr, cmap="YlGnBu", annot=True)

# displaying heatmap
#figure = dataplot.get_figure()    
#figure.savefig('corr_heatmap.png', dpi=400)
plt.show()

In [ ]:
features=corr[corr['SalePrice']>0.3].index.tolist()
features=features[:-1]
features

In [ ]:
xtrain=X[features]
xtest=xtest[features]
xtrain.isnull().sum()


In [ ]:
#LotFrontage null values to be replaced by mean, 
#as it has less outliers so we can impute accordingly
plt.hist(xtrain['LotFrontage'],bins=20)

In [ ]:
xtrain.loc[:]['LotFrontage'].fillna(xtrain['LotFrontage'].mean(),inplace=True)
xtest.loc[:]['LotFrontage'].fillna(xtrain['LotFrontage'].mean(),inplace=True)
xtrain.isnull().sum()

In [ ]:
#Garage year built to be replaced by median as it has more outliers
plt.hist(xtrain['GarageYrBlt'])

In [ ]:
xtrain.loc[:]['GarageYrBlt'].fillna(xtrain['GarageYrBlt'].median(),inplace=True)
xtest.loc[:]['GarageYrBlt'].fillna(xtrain['GarageYrBlt'].median(),inplace=True)
xtrain.isnull().sum()

In [ ]:
xtest.fillna(0,inplace=True)

In [ ]:
#Replacing the missing values with mode
plt.hist(xtrain['MasVnrArea'],bins=20)


In [ ]:
xtrain.loc[:]['MasVnrArea'].fillna(xtrain['MasVnrArea'].mode()[0],inplace=True)
xtest.loc[:]['MasVnrArea'].fillna(xtrain['MasVnrArea'].mode()[0],inplace=True)
xtrain.isnull().sum()

In [ ]:
def evaluation_metrics(xtrain,ytrain,model):
    model.fit(xtrain,ytrain)
    cv = KFold(n_splits=10, random_state=1, shuffle=True)
    scores = cross_val_score(model, xtrain, ytrain, scoring='d2_absolute_error_score', cv=cv, n_jobs=-1)
    # report performance
    print(f'Scoring  using KFold: {np.mean(scores)}')
    print(f'Training Score for {model} : {model.score(xtrain,ytrain):.2f}')

In [ ]:
lr=LinearRegression()
br=BayesianRidge()
sgdr=SGDRegressor()
en=ElasticNet()
gbr=GradientBoostingRegressor
svr=SVR()
kr=KernelRidge()
xgb=XGBRegressor
cgb=CatBoostRegressor()
lgbm=LGBMRegressor()

In [ ]:
models=[lr,br,kr,cgb,lgbm]

In [ ]:
for model in models:
    evaluation_metrics(xtrain,y,model)

In [ ]:
xtest.isnull().sum()

In [ ]:
results=cgb.predict(xtest)
results=pd.DataFrame({'Id':ids,'SalePrice':results})
results.to_csv('submission.csv',index=False)